# Voice Assistant Using ChatGPT and Whisper APIs

Import necessary libraries

In [1]:
import gradio as gr
import openai
import config
import subprocess
from pydub import AudioSegment

Set OpenAI API key and Define initial system message

In [2]:
openai.api_key = config.OPENAI_API_KEY
messages = [{"role": "system", "content": 'You are a Helpful assistant, respond in 30 words or less'}]

Define the transcribe function that transcribes audio using OpenAI

In [3]:
def transcribe(audio):
    global messages

    audio_file_wav = open(audio, "rb")
    audio_file_mp3 = AudioSegment.from_wav(audio_file_wav).export("audio.mp3", format="mp3")
    transcript = openai.Audio.transcribe("whisper-1", audio_file_mp3)

    messages.append({"role": "user", "content": transcript["text"]})

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

    system_message = response["choices"][0]["message"]
    messages.append(system_message)

    subprocess.call(["say", system_message['content']])

    chat_transcript = ""
    for message in messages:
        if message['role'] != 'system':
            chat_transcript += message['role'] + ": " + message['content'] + "\n\n"

    return chat_transcript

Create a Gradio interface for the transcribe function

In [4]:
ui = gr.Interface(fn=transcribe, inputs=gr.Audio(source="microphone", type="filepath"), outputs="text")
ui.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/lasso/Documents/jupyter-server/voice-gpt/venv/lib/python3.10/site-packages/gradio/processing_utils.py:239: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
